In [9]:
from pdfminer import high_level
import PIL

In [39]:
path = r'C:\Users\Yatharth\Documents\VacCert-Project\cert.pdf'
text = high_level.extract_text(path)
print(text)

In [49]:
info = ["name", "age", "sex", "verif", "uhid", "refID", "vax", "dose1", "dose2"]
info_dict = {}
#print(info)

In [50]:
ls = text.split("\n\n")
#print(ls)

['Yatharth Sood', '21', 'Male', 'Aadhaar # XXXXXXXX0468', '61-8613-2248-6680', '93409364295360', 'COVISHIELD', '27 May 2021 (Batch no. 4121Z070)', '21 Aug 2021 (Batch no. 4121MC057)', 'Amar', 'Radha Swami Satsang Solan, Solan,', 'Himachal Pradesh', 'Final Certiﬁcate for COVID-19 VaccinationBeneﬁciary DetailsVaccination DetailsBeneﬁciary Name /Gender /Age /ID Veriﬁed /Unique Health ID (UHID)Beneﬁciary Reference IDVaccine Name /Date of 1st Dose /Date of 2nd Dose /Vaccinated by /Vaccination at /लाभाथ\x00 का नामउ\x00\x00ल\x00गपहचान प\x00 स\x00ा\x00पतवै\x00ीन का नामपहली खुराक क\x00 तारीख\x00ूसरी खुराक क\x00 तारीखटीका लगाने वाले का नामटीकाकरण का \x00ानThis certiﬁcate can be veriﬁed by scanning the QR code at http://verify.cowin.gov.inTogether, India will defeat COVID-19”In case of any adverse events, kindly contact the nearest Public Health Center/Healthcare Worker/District Immunization Oﬃcer/State Helpline No. 1075“दवाई भी और कड़ाई भी।टीकाकरण प\x00ात \x00कसी \x00\x00तकूल घटना के होने पर नज़दी

In [51]:
for i in range(len(info)):
    info_dict[info[i]] = ls[i]
print(info_dict)

{'name': 'Yatharth Sood', 'age': '21', 'sex': 'Male', 'verif': 'Aadhaar # XXXXXXXX0468', 'uhid': '61-8613-2248-6680', 'refID': '93409364295360', 'vax': 'COVISHIELD', 'dose1': '27 May 2021 (Batch no. 4121Z070)', 'dose2': '21 Aug 2021 (Batch no. 4121MC057)'}
